In [7]:
import pandas as pd
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

train_data_file = './train_data02.csv'
test_data_file = './val_data02.csv'
train_data = pd.read_csv(train_data_file, encoding='utf-8')
test_data = pd.read_csv(test_data_file, encoding='utf-8')

In [8]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', 10)
train_data.head()
#print(train_data.shape)
#print(test_data.shape)

,pax_name,pax_passport,seg_route_to,seg_flight,seg_cabin,emd_lable,emd_lable2,gender,age,residence_country,nation_name,province_name,member_level,enroll_chnl,pref_aircraft_m3_1,pref_aircraft_m3_2,pref_aircraft_y3_1,pref_aircraft_y3_2,pref_aircraft_y3_3,pref_aircraft_y3_4,pref_aircraft_y3_5,pref_line_y3_1,pref_line_y3_2,pref_line_y3_3,pref_line_y3_4,pref_line_y3_5,pref_city_y3_1,pref_city_y3_2,pref_city_y3_3,pref_dest_city_y1,pref_dest_city_y2,pref_dest_city_y3,pit_add_chnl_y3,seat_walkway_cnt_y3,seat_window_cnt_y3,seat_middle_cnt_y3,avg_dist_cnt_y2,avg_dist_cnt_y3,flt_cnt_y3,flt_leg_i_cnt_y3,flt_leg_d_cnt_y3,mdl_mcv,tkt_avg_amt_y1,tkt_avg_amt_y2,tkt_avg_amt_y3
0,9612c51f08718e48b1361e415ef55be9,93d0aceeaef3f85264e5c5b5b1148a90,0.0,6.0,2.0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0.000000
1,c7ca533cd9da4c4bf3d597248f90bc6e,6930f6fa59d86c11fa28c610ab6f56a5,0.0,6.0,2.0,1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,276.0,317.0,246.0,0.0,0.0,0.0,0.0,0.0,58.0,55.0,52.0,0.0,0,2,0,4986.666667,4986.666667,3,0,1,50,2950.223333,3253.556667,3253.556667
2,bd2cb6b0dd56fb3de6f7c018296a07e2,da73afa8f6ededd29d67a9331dccdbe9,1.0,9.0,4.0,1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0.000000
3,65657872fe131ea7651565638dcb5683,dc31eead23771810dbb586ad236ade3b,1.0,9.0,4.0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,24.0,27.0,28.0,26.0,25.0,89.0,223.0,0.0,0.0,0.0,43.0,58.0,0.0,0.0,0.0,41.0,0.0,1,1,0,10808.000000,10837.500000,2,2,0,50,0.000000,0.000000,4515.030000
4,722261f7b860b183da9fe0b6630b6dbf,7689603247539830a08d945cb1a57823,0.0,10.0,4.0,1,1,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,24.0,9.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.0,46.0,58.0,0.0,0.0,0.0,0.0,4,0,0,0.000000,0.000000,0,4,0,0,0.000000,0.000000,0.000000


In [9]:
features_columns = [col for col in train_data.columns if col not in ['pax_name', 'pax_passport', 'emd_lable2']]
#features_columns_test = [col for col in test_data.columns ]

train = train_data[features_columns].values
target =train_data['emd_lable2'].values

#需要提交的测试集
#test = test_data[features_columns].values
#target_test = test_data['emd_lable2'].values

In [10]:
from sklearn.model_selection import train_test_split
import lightgbm
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.4, random_state=0)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.5, random_state=0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
#print(test.shape, target_test.shape)

clf = lightgbm

train_matrix = clf.Dataset(X_train, label=y_train)
test_matrix = clf.Dataset(X_test, label=y_test)
params = {
          'boosting_type': 'gbdt',
          #'boosting_type': 'dart',
          'objective': 'multiclass',
          'metric': 'multi_logloss',
          #'metric': 'auc',
          'min_child_weight': 1.5,
          'max_depth': 10,         #树的最大深度
          'num_leaves': 40,        #取值应 <= 2 ^（max_depth）， 超过此值会导致过拟合
          'lambda_l2': 10,
          'subsample': 0.7,
          'colsample_bytree': 0.7,
          'colsample_bylevel': 0.7,
          'learning_rate': 0.01,   #学习率
          'tree_method': 'exact',
          'seed': 2017,
          "num_class": 2,          #分类数
          'silent': True,         #静默模式开启，不会输出任何信息。
          }
num_round = 3600
early_stopping_rounds = 100
model = clf.train(params, 
                  train_matrix,
                  num_round,
                  valid_sets=test_matrix,
                  early_stopping_rounds=early_stopping_rounds)
pre= model.predict(X_valid,num_iteration=model.best_iteration)
#print(pre)

(14059, 42) (14059,)
(4686, 42) (4686,)
[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002863 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3168
[LightGBM] [Info] Number of data points in the train set: 14059, number of used features: 42
[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Info] Start training from score -0.066613
[LightGBM] [Info] Start training from score -2.741979
[LightGBM] [Warning] No f

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1077: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[18]	valid_0's multi_logloss: 0.201179
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[19]	valid_0's multi_logloss: 0.200024
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[20]	valid_0's multi_logloss: 0.198832
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[21]	valid_0's multi_logloss: 0.198151
[22]	valid_0's multi_logloss: 0.197908
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[23]	valid_0's multi_logloss: 0.196835
[24]	valid_0's multi_logloss: 0.196727
[25]	valid_0's multi_logloss: 0.196499
[LightGBM] [

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[87]	valid_0's multi_logloss: 0.162613
[88]	valid_0's multi_logloss: 0.16235
[89]	valid_0's multi_logloss: 0.162251
[90]	valid_0's multi_logloss: 0.162136
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[91]	valid_0's multi_logloss: 0.161681
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[92]	valid_0's multi_logloss: 0.161257
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[93]	valid_0's multi_logloss: 0.16087
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[94]	valid_0's multi_logloss: 0.160614
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [War

[165]	valid_0's multi_logloss: 0.144921
[166]	valid_0's multi_logloss: 0.144804
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[167]	valid_0's multi_logloss: 0.144689
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[168]	valid_0's multi_logloss: 0.144497
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[169]	valid_0's multi_logloss: 0.144297
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[170]	valid_0's multi_logloss: 0.144106
[171]	valid_0's multi_logloss: 0.144063
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[172]	valid_0's multi_logloss: 0.143892
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[173]

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[723]	valid_0's multi_logloss: 0.126459
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[724]	valid_0's multi_logloss: 0.12646
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[725]	valid_0's multi_logloss: 0.126456
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[726]	valid_0's multi_logloss: 0.126451
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[727]	valid_0's multi_logloss: 0.126446
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[728]	valid_0's multi_logloss: 0.126451
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[729]	valid

[783]	valid_0's multi_logloss: 0.126243
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[784]	valid_0's multi_logloss: 0.126242
[785]	valid_0's multi_logloss: 0.12624
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[786]	valid_0's multi_logloss: 0.126234
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[787]	valid_0's multi_logloss: 0.126232
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[788]	valid_0's multi_logloss: 0.126236
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[789]	valid_0's multi_logloss: 0.126232
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[790]	valid_0's multi_logloss: 0.126234
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[791]	

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[851]	valid_0's multi_logloss: 0.1261
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[852]	valid_0's multi_logloss: 0.1261
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[853]	valid_0's multi_logloss: 0.126095
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[854]	valid_0's multi_logloss: 0.126091
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[855]	valid_0's multi_logloss: 0.126091
[856]	valid_0's multi_logloss: 0.126092
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[857]	valid_0's multi_logloss: 0.126089
[LightGBM] [Warning] No further splits with posit

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[930]	valid_0's multi_logloss: 0.126007
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[931]	valid_0's multi_logloss: 0.126007
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[932]	valid_0's multi_logloss: 0.126008
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[933]	valid_0's multi_logloss: 0.126001
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[934]	valid_0's multi_logloss: 0.126003
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[935]	valid_0's multi_logloss: 0.126001
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[936]	valid_0's multi_logloss: 0.126002
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[988]	valid_0's multi_logloss: 0.126031
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[989]	valid_0's multi_logloss: 0.126029
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[990]	valid_0's multi_logloss: 0.126028
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[991]	valid_0's multi_logloss: 0.126028
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[992]	valid_0's multi_logloss: 0.126037
[993]	valid_0's multi_logloss: 0.126037
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[994]	vali

In [11]:
print('score : ', np.mean((pre[:,1]>0.50)==y_valid))

score :  0.9415404309793045


In [49]:
def find_repeat_data(name_list):
    """
    查找列表中重复的数据
    :param name_list: 
    :return: 一个重复数据的列表，列表中字典的key 是重复的数据，value 是重复的次数
    """
    repeat_list = []
    for i in set(name_list):
        ret=name_list.count(i) # 查找该数据在原列表中的个数
        if ret > 1:
            item=dict()
            item[i] = ret
            repeat_list.append(item)
    print(repeat_list)
    return repeat_list

find_repeat_data(list(y_test))
find_repeat_data(list(predicted))
find_repeat_data(list(target_test))

[{0: 8804}, {1: 569}]
[{0: 9206}, {1: 167}]
[{0: 6771}]


[{0: 6771}]

In [26]:
print(type(data1))
#print(data2)
df = pd.DataFrame(data1)
df.insert(2, 'answer', list(y_test))  # 2表示插入列的位置（索引）， 'answer'是列标题
print(type(df))
print(df)
df.to_csv('output_classifier.csv')

<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>
             0         1  answer
0     0.998538  0.001462       0
1     0.993159  0.006841       0
2     0.959522  0.040478       0
3     0.759812  0.240188       0
4     0.757695  0.242305       0
...        ...       ...     ...
9368  0.768289  0.231711       0
9369  0.992677  0.007323       0
9370  0.996305  0.003695       0
9371  0.993816  0.006184       0
9372  0.997400  0.002600       0

[9373 rows x 3 columns]


In [50]:
print(type(data2))
#print(data2)
df = pd.DataFrame(data2)
df.insert(2, 'answer', list(target_test))  # 2表示插入列的位置（索引）， 'answer'是列标题
print(type(df))
print(df)
df.to_csv('output_classifier_test.csv')

<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>
             0         1  answer
0     0.994000  0.006000       0
1     0.984000  0.016000       0
2     0.995807  0.004193       0
3     0.989884  0.010116       0
4     0.995893  0.004107       0
...        ...       ...     ...
6766  0.984000  0.016000       0
6767  0.984000  0.016000       0
6768  0.995893  0.004107       0
6769  0.997893  0.002107       0
6770  0.984000  0.016000       0

[6771 rows x 3 columns]
